In [28]:
import sqlite3
import pandas as pd

# Create your connection.
# DL : https://drive.google.com/file/d/10CtW_gZtNHQbQQ2A7QRiXXTh4obXHOQk/view?usp=sharing
con = sqlite3.connect('batch_1.db')
cursorObj = con.cursor()
cursorObj.execute('SELECT name from sqlite_master where type= "table"')
print(cursorObj.fetchall())

[('aeroports',), ('compagnies',), ('vols',)]


In [42]:
aeroports = pd.read_sql_query("SELECT * FROM aeroports", con)
aeroports = aeroports.rename(columns={
    'CODE IATA': 'CODE_IATA',
    'PRIX RETARD PREMIERE 10 MINUTES':'PRIX_RETARD_10',
    'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES':'PRIX_RETARD_APRES'
})
aeroports.to_csv("aeroports.csv", index=False)
aeroports.sort_values(by="CODE_IATA").head()

,CODE_IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX_RETARD_10,PRIX_RETARD_APRES
215,AAL,Aalborg Airport,Aalborg,DK,9.84924316406,57.0927589138,10.0,8346,353
201,AAP,Aji Pangeran Tumenggung Pranoto International ...,Samarinda,ID,117.249392,-0.374448,73.0,9245,318
46,ABV,Nnamdi Azikiwe International Airport,Abuja,NG,7.263169765472412,9.006790161132812,1123.0,6338,272
247,ACC,Kotoka International Airport,Accra,GH,-0.16678600013256073,5.605189800262451,205.0,2880,955
254,ADB,Adnan Menderes International Airport,Ä°zmir,TR,27.156999588,38.2924003601,412.0,2955,603


In [30]:
compagnies = pd.read_sql_query("SELECT * FROM compagnies", con)
compagnies.to_csv("compagnies.csv", index=False)

In [31]:
vols = pd.read_sql_query("SELECT * FROM vols", con)
vols = vols.rename(columns={
'CODE AVION':'CODE_AVION',
'AEROPORT DEPART':'AEROPORT_DEPART',
'AEROPORT ARRIVEE':'AEROPORT_ARRIVEE',
'DEPART PROGRAMME':'HEURE_DEPART_PROGRAMME',
'HEURE DE DEPART':'HEURE_DEPART',
'RETART DE DEPART':'TEMPS_RETARD_DEPART',
'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE':'TEMPS_DEPLACEMENT_DECOLLAGE',
'DECOLLAGE':'HEURE_DECOLLAGE',
'TEMPS PROGRAMME':'TEMPS_PROGRAMME',
'TEMPS PASSE':'TEMPS_PASSE',
'TEMPS DE VOL':'TEMPS_VOL',
'DISTANCE':'DISTANCE',
'ATTERRISSAGE':'HEURE_ATTERRISSAGE',
"TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE":'TEMPS_DEPLACEMENT_ATTERRISSAGE',
'ARRIVEE PROGRAMMEE':'HEURE_ARRIVEE_PROGRAMMEE',
"HEURE D'ARRIVEE":'HEURE_ARRIVEE',
"RETARD A L'ARRIVEE":'TEMPS_RETARD_ARRIVEE',
'DETOURNEMENT':'DETOURNEMENT',
'ANNULATION':'ANNULATION',
'DATE':'DATE',
'NIVEAU DE SECURITE':'NIVEAU_SECURITE',
#'COMPAGNIE AERIENNE':'COMPAGNIE',
"RAISON D'ANNULATION":"RAISON_ANNULATION",
"RETARD SYSTEM":"RETARD_SYSTEM",
"RETARD SECURITE":"RETARD_SECURITE",
"RETARD COMPAGNIE":"RETARD_COMPAGNIE",
"RETARD AVION":"RETARD_AVION",
"RETARD METEO":"RETARD_METEO"
})
vols.to_csv("vols.csv", index=False)

In [32]:
aeroports = aeroports.drop_duplicates(subset=['CODE_IATA'])
airports_start = aeroports.add_prefix('DEPART_')
airports_end = aeroports.add_prefix('ARRIVEE_')

# Add companies
vols_and_companies = vols.join(compagnies.set_index(['CODE']),
                               on=['COMPAGNIE AERIENNE'],
                               how='left')
print(vols_and_companies.shape)

(3000000, 31)


In [53]:
aeroports[aeroports['CODE_IATA'] == 'DSS']

,CODE_IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX_RETARD_10,PRIX_RETARD_APRES
92,DSS,Blaise Diagne International Airport,Dakar,SN,-17.073333,14.67,290.0,5499,120


In [33]:
# Add airport departure information
vols_and_companies_departure = vols_and_companies.join(airports_start.set_index(
    ['DEPART_CODE_IATA']),
    on=['AEROPORT_DEPART'],
    how='inner')

In [49]:
# Add airport arrival information
full_df = vols_and_companies_departure.join(airports_end.set_index(
    ['ARRIVEE_CODE_IATA']),
    on=['AEROPORT_ARRIVEE'],
    how='inner')

In [55]:
full_df.to_csv("full_join.csv", index=False)

In [54]:
full_df[full_df['AEROPORT_DEPART'] == 'AAL'].head()[['AEROPORT_DEPART','DEPART_PRIX_RETARD_10','DEPART_PRIX_RETARD_APRES','ARRIVEE_PRIX_RETARD_10','ARRIVEE_PRIX_RETARD_APRES','AEROPORT_ARRIVEE']]

,AEROPORT_DEPART,DEPART_PRIX_RETARD_10,DEPART_PRIX_RETARD_APRES,ARRIVEE_PRIX_RETARD_10,ARRIVEE_PRIX_RETARD_APRES,AEROPORT_ARRIVEE
3155,AAL,8346,353,5499,120,DSS
5491,AAL,8346,353,5499,120,DSS
6862,AAL,8346,353,5499,120,DSS
8431,AAL,8346,353,5499,120,DSS
14048,AAL,8346,353,5499,120,DSS


In [37]:
aeroports = aeroports.drop_duplicates(subset=['CODE_IATA'])
vols_depart = vols[['AEROPORT_DEPART',
       'HEURE_DEPART_PROGRAMME', 'HEURE_DEPART',
       'TEMPS_RETARD_DEPART', 'TEMPS_DEPLACEMENT_DECOLLAGE', 'HEURE_DECOLLAGE',
       'TEMPS_PROGRAMME', 'TEMPS_PASSE', 'TEMPS_VOL', 'DISTANCE',
       
       'DETOURNEMENT', 'ANNULATION', 'RAISON_ANNULATION', 'RETARD_SYSTEM',
       'RETARD_SECURITE', 'RETARD_COMPAGNIE', 'RETARD_AVION', 'RETARD_METEO',
       'DATE', 'NIVEAU_SECURITE', 'COMPAGNIE AERIENNE']]

,CODE_IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX_RETARD_10,PRIX_RETARD_APRES
0,MCT,Muscat International Airport,Muscat,OM,58.284400939941406,23.593299865722656,48.0,6695,361
1,SOU,Southampton Airport,Southampton,GB,-1.3567999601364136,50.95029830932617,44.0,1061,71
2,PNH,Phnom Penh International Airport,Phnom Penh,KH,104.84400177001953,11.546600341796875,40.0,2402,909
3,BLR,Kempegowda International Airport,Bangalore,IN,77.706299,13.1979,3000.0,8641,550
4,FFD,RAF Fairford,Fairford,GB,-1.7900300025900002,51.6822013855,286.0,6883,870
5,CGO,Zhengzhou Xinzheng International Airport,Zhengzhou,CN,113.841003418,34.519699096699995,495.0,4753,117
6,DRS,Dresden Airport,Dresden,DE,13.767200469970703,51.1328010559082,755.0,1408,183
7,BZZ,RAF Brize Norton,Brize Norton,GB,-1.58362,51.75,288.0,3252,197
8,MWX,Muan International Airport,Piseo-ri (Muan),KR,126.382814,34.991406,35.0,3140,517
9,DAR,Julius Nyerere International Airport,Dar es Salaam,TZ,39.202599,-6.87811,182.0,3615,109


In [40]:
vols.columns

Index(['IDENTIFIANT', 'VOL', 'CODE_AVION', 'AEROPORT_DEPART',
       'AEROPORT_ARRIVEE', 'HEURE_DEPART_PROGRAMME', 'HEURE_DEPART',
       'TEMPS_RETARD_DEPART', 'TEMPS_DEPLACEMENT_DECOLLAGE', 'HEURE_DECOLLAGE',
       'TEMPS_PROGRAMME', 'TEMPS_PASSE', 'TEMPS_VOL', 'DISTANCE',
       'HEURE_ATTERRISSAGE', 'TEMPS_DEPLACEMENT_ATTERRISSAGE',
       'HEURE_ARRIVEE_PROGRAMMEE', 'HEURE_ARRIVEE', 'TEMPS_RETARD_ARRIVEE',
       'DETOURNEMENT', 'ANNULATION', 'RAISON_ANNULATION', 'RETARD_SYSTEM',
       'RETARD_SECURITE', 'RETARD_COMPAGNIE', 'RETARD_AVION', 'RETARD_METEO',
       'DATE', 'NIVEAU_SECURITE', 'COMPAGNIE AERIENNE'],
      dtype='object')